In [1]:
from base import np

class Solver1D:
    def __init__(self, params, c_func, n0_func = None):
        """
        Solves the 1D Diffusion with Consumption problem using the Finite Difference Method.
        
        Args:
            params (dict): System parameters.
            c_func (callable): Bacterial distribution function.
            n0_func (callable, optional): Initial condition for the Nutrients Concentration.
        """
        # System Constants
        self.L  = params['L']
        self.T  = params['T']
        self.Tc = params['Tc']
        # self.Td = params['Td']
        # self.Nb = params['Nb']
        # # self.D  = self.L**2 / (2 * self.Td)
        # self.D  = self.L**2 / self.Td
        self.alpha = 1.0 / self.Tc
        # Discretisation Constants
        self.nx = params['nx']
        self.nt = params['nt']

        self.__discretise_system(n0_func, c_func)
        

    def __discretise_system(self, n0_func, c_func):
        """Discretise Space/Time and input distributions."""
        self.dx = self.L / (self.nx - 1)
        self.x = np.linspace(0, self.L, self.nx)
        self.t = np.linspace(0, self.T, self.nt)
        self.c = c_func(self.x)
        self.c_tag = c_func.__name__.split('_')[1]
        if n0_func is not None:
            self.n0 = n0_func(self.x)

    def _stability_condition(self):
        """Calculate the stability condition for the PDE, Runge-Kutta 5(4)."""
        # Eigenvalue from discretisation
        R = 1.6 # Stability factor for RK5(4)
        lambd = - 2 / self.dx**2 #- self.alpha * np.max(self.c)
        dt = R / abs(lambd)
        nt = int(self.T / dt + 1)
        return nt

In [31]:
syst_params = {
'L' : 1.0,  'T' : 1.0, # Lenght & Time Domain
'Tc': 1.0,             # Consumption time
'nx': 10,   'nt': 100  # Num Spatial/Temporal points
}
L = syst_params['L']

# Initial condition for the nutrients
n0_linear = lambda x: x / L
# Bacterium distribution functions
def c_const(x):
    return np.ones_like(x)
def c_midstep(x):
    x0 = L/3 # Starting point of the step
    l  = L/3 # Length of the step
    cond = (x >= x0) & (x <= x0 + l)
    return np.where( cond , 1/l , 0)
def c_exp(x):
    a = 1
    return a/L * np.exp( a*(1-x/L) ) / (np.exp(a) - 1)

S1D = Solver1D(syst_params, c_exp, n0_linear)

print('nt=',S1D._stability_condition())

nt= 102


In [28]:
lambd = - 2 / S1D.dx**2 - S1D.alpha * np.max(S1D.c)
dt = 1.6 / abs(lambd)
dt

0.0003330847704397649

In [27]:
dx = S1D.L / (S1D.nx - 1)
dx

0.02040816326530612